# Using Papermill to Register Jupyter Notebook Model in Dojo

The first step to run a model in a jupyter noteook from the command line with Papermill is to determine how you are setting all your model parameters. 

The options are mainly, from config/parameter files that are read in or as a flag. 

If you model reads the parameters in from a config file you don't need to do anything with the parameters. 

If they are passed to your model as a flag then you will need to tag those in the notebook. To do this you need to tag the cell in your notebook where you define your parameters and have them set to appropriate default values.


### If you are using a flag to set a parameter please continue ... 

## Jupyter Notebook
Naviagate to View/Cell Toolbar/Tags to add the ability to add a tag to a cell. 



## If using JupyterLab v3 or above:

Select the cell to parameterize

Click the property inspector in the right sidebar (double gear icon)

Type “parameters” in the “Add Tag” box and hit “Enter”.

## JupyterLab 2.0 - 2.2.x
If using the JupyterLab interface

Select the cell to parameterize

Click the property inspector on the left side (double gear icon)

Type “parameters” in the “Add Tag” box and hit “Enter”.

## Dummy example
In this dummy model parameters are read in from the two config files, fakeDataType.json and parameters.json.However, one parameter *temp is set via a command line --flag. You can see where in the code the parameters are read in below. 

```
params = open('configFiles/parameters.json').read()
params = json.loads(params)

configFileData = open('configFiles/fakeDataType.json').read()
configFileData = json.loads(configFileData)
```

Papermill will run the notebook with whatever values are set for the parameters in the config files, so if you tag these files as parameters with built in command line tools they will be exposed correctly and Papermill will run the model with the updated parameter values every run. 

### working with --flag parameters

flag parameters are parameters that are set on the command line when running the model. This dummy model has a temp parameter which is a flag parameter. Here is what is needed to set temp to .2 for a model run on a python script, but this doesn't work for a notebook unless we add a parameter tag to a cell with our temp variable. 
```
python main.py --temp .2

```



In [1]:
# set temp default to what your models default run should be. 
temp=.5

In [2]:
import click
import random
import json
import time
import pandas as pd
from faker import Faker
from collections import OrderedDict
import urllib.request
import os
import requests
if not os.path.exists('output'):
    os.mkdir('output')

if not os.path.exists('media'):
    os.mkdir('media')    

In [6]:
params = open('configFiles/parameters.json').read()
params = json.loads(params)

configFileData = open('configFiles/fakeDataType.json').read()
configFileData = json.loads(configFileData)


print('Beginning to download file from S3...')

url = 'https://jataware-world-modelers.s3.amazonaws.com/dummy-model/input.csv'
urllib.request.urlretrieve(url, 'input.csv')


locales = OrderedDict([
    ('en-US', 1)
])
fake = Faker(locales)
f = fake['en_US']    
data = []

perturbation = 1 + (params['rainfall']*temp)
getColorHue = configFileData['color_hue']
print('print params')
print(f'color {getColorHue}')
print(params['rainfall'])
print(f'temp {temp}')
print('done with getting params')
for i in range(100):
    obj = dict(latitude=f.latitude(),
               longitude=f.longitude(),
               date=f.date(),
               value=random.random()*perturbation,
               color_hue=f.color(hue=getColorHue))
    data.append(obj)

df = pd.DataFrame(data)
print(df.head())
fname = f"output_{params['rainfall']}_{temp}"
df.to_csv(f'output/{fname}.csv', index=False)
print(f"Saved output as /model/output/{fname}.csv")

Beginning to download file from S3...
print params
color blue
0.9
temp 0.5
done with getting params
      latitude    longitude        date     value color_hue
0    22.733040  -105.950221  1994-06-04  0.701154   #a2a1e2
1    5.2256655  -168.387601  2017-04-26  0.305415   #4c39a0
2  -59.5166715   -15.007695  1986-07-20  1.347375   #aee8f9
3    4.0917175    -4.288002  2018-07-26  1.121521   #130d84
4    55.672092    58.406969  1974-09-11  0.799305   #52d4d8
Saved output as /model/output/output_0.9_0.5.csv


In [4]:
def get_media():
    image_urls = ['https://i.kym-cdn.com/entries/icons/facebook/000/000/774/lime-cat.jpg',
                  'https://pbs.twimg.com/profile_images/1356088845821857795/1WWMDwIQ_400x400.jpg']
    for url in image_urls:
        filename = url.split('/')[-1]
        img_data = requests.get(url).content
        with open(f'media/{filename}', 'wb') as handler:
            handler.write(img_data)
        print(f"Wrote media: {filename}")
    return
get_media()

/home/kyle/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


Wrote media: lime-cat.jpg
Wrote media: 1WWMDwIQ_400x400.jpg
